### KPC C09K11/06
* KPC 설계 검증용 데이터를 출원일자 기준으로 분할하여, multi-label 학습 데이터 생성

In [1]:
import time
%time

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 2.38 µs


In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import pickle

### Load Data

In [3]:
# 엑셀 파일의 모든 시트명 읽어와 필요한 시트만 다시 읽기
data = pd.read_excel('kpc_c09k/KPC_C09K11_재분류(수정분류표기반).xlsx', sheet_name=None)
sheets_list = list(data.keys())[1:]
data = pd.read_excel('kpc_c09k/KPC_C09K11_재분류(수정분류표기반).xlsx', sheet_name=sheets_list)

In [4]:
# 하나의 dataframe으로 합치기
data_total = pd.DataFrame()
count = {}
for nm in sheets_list:
    temp = data[nm]
    count[nm] = len(temp)
    temp['label'] = nm
    if len(data_total) > 0:
        data_total = pd.concat([data_total, temp])
    else:
        data_total = temp

In [5]:
# count
sum(count.values())  # 1369

1369

In [6]:
data_total.columns
# Index(['연번', '출원번호', '출원일', '공개번호/\n공고번호', '문서\n종류', '공개일/\n공고일', '제목', '출원인',
# 'CPC 코드', 'KPC 코드', 'label', '비  고', 'Unnamed: 11'], dtype='object')

Index(['연번', '출원번호', '출원일', '공개번호/\n공고번호', '문서\n종류', '공개일/\n공고일', '제목', '출원인',
       'CPC 코드', 'KPC 코드', 'label', '비  고', 'Unnamed: 11'],
      dtype='object')

#### test 데이터 구분
* 출원일이 20200400 이후의 출원번호를 테스트, 이전은 학습 데이터로 사용 (공개/등록건이 일괄 구분되도록)

In [ ]:
test_split_date = 20200400
test_data = data_total.loc[data_total['출원일'] >= test_split_date]
test_count = test_data.groupby(['label'])['출원번호'].count()
test_count = dict(test_count)
# print(sum(test_count.values()), test_count)  # 473 
share = {}
for label in count.keys():
    if label in test_count.keys():
        share[label] = test_count[label] / count[label]
    else:
        share[label] = 0
# share

In [ ]:
# 공개일로 구분 테스트했던 코드(삭제 예정)
# test_data = data_total.loc[data_total['공개일/\n공고일'] >= 20210600]
# test_count = test_data.groupby(['label'])['출원번호'].count()
# test_count = dict(test_count)
# print(sum(test_count.values()), test_count)  # 395 

#### 출원번호의 중복을 제거하여 서지사항 검색용 검색식 생성

In [ ]:
# 출원번호의 중복을 제거하고 검색식 만들기
len(data_total)  # 1369
appln_no_uniq = data_total['출원번호'].unique()
len(appln_no_uniq)  # 665
# print(', '.join(appln_no_uniq))

#### 출원번호 - 라벨 쌍 생성

In [ ]:
label_data = data_total[['출원번호', '출원일', 'label']]
label_data.columns = ['appln_no', 'ad', 'label']
label_data[:2]

#### 검색결과 서지사항에서 텍스트 추출

In [ ]:
biblio = pd.read_excel('kpc_c09k/c09k_861_biblio.xlsx')
# biblio[:2]

In [ ]:
biblio.columns

In [ ]:
text_columns = ['발명의명칭', '요약', '대표청구항', '과제', '해결방안']
replace_strings = ['\t', '@', '\n']

In [ ]:
for col in text_columns:
    for str_ in replace_strings:
        biblio[col] = biblio[col].apply(lambda x: x.replace(str_, " "))

In [ ]:
biblio['tl_ab'] = biblio[['발명의명칭', '요약']].apply(lambda x: ' '.join(x), axis=1)
biblio['tl_cl'] = biblio[['발명의명칭', '대표청구항']].apply(lambda x: ' '.join(x), axis=1)
biblio['tl_pu'] = biblio[['발명의명칭', '과제']].apply(lambda x: ' '.join(x), axis=1)
biblio['tl_so'] = biblio[['발명의명칭', '해결방안']].apply(lambda x: ' '.join(x), axis=1)
text_columns.extend(['tl_ab', 'tl_cl', 'tl_pu', 'tl_so'])

In [ ]:
# biblio.loc[biblio['대표청구항'].str.len() < 10]  # 대표청구항이 삭제된건 : 총 10건
# # 출원번호 : KR102020000043867, KR102020000004267, ...

In [ ]:
# 10글자 이하의 텍스트는 학습에서 제외
# applnno_text = pd.DataFrame(columns=['appln_no', 'text'])

# for cols in text_columns:
#     tmp = biblio[['출원번호', cols]]
#     tmp.columns = ['appln_no', 'text']
#     applnno_text = pd.concat([applnno_text, tmp], axis=0, ignore_index=True)

# applnno_text[:2]  # 7749 rows × 2 columns

In [ ]:
# 10글자 이하의 텍스트는 학습에서 제외
applnno_text = pd.DataFrame(columns=['appln_no', 'text'])

for cols in text_columns:
    tmp = biblio[['출원번호', cols]].loc[biblio[cols].str.len() > 10]
    tmp.columns = ['appln_no', 'text']
    applnno_text = pd.concat([applnno_text, tmp], axis=0, ignore_index=True)

applnno_text[:2]  # 7676 rows × 2 columns
# text_abst = biblio[['출원번호', '요약']]
# text_cl = biblio[['출원번호', '대표청구항']]
# text_purpose = biblio[['출원번호', '과제']]
# text_solution = biblio[['출원번호', '해결방안']]
# text_title.columns = ['appln_no', 'text']
# text_abst.columns = ['appln_no', 'text']
# text_cl.columns = ['appln_no', 'text']
# text_purpose.columns = ['appln_no', 'text']
# text_solution.columns = ['appln_no', 'text']

#### 라벨과 텍스트 데이터 병합

In [ ]:
# label_text = pd.DataFrame(columns=['text', 'label'])
# label_text
# def extract_label_text(text_df, label_df):
#     temp = pd.merge(left=label_data, right=text_title, left_index=False, right_index=False, how='inner', on='appln_no')

In [ ]:
label_text = pd.merge(left=label_data, right=applnno_text, left_index=False, right_index=False, how='inner', on='appln_no')
label_text

In [ ]:
kpc_label = label_text['label'].unique()
kpc_index_dict = {i: label for i, label in enumerate(kpc_label)}
kpc_label_dict = {kpc_index_dict[key]: key for key in kpc_index_dict.keys()}
with open('kpc_c09k/c09k_ind_label.pickle', 'wb') as f:
    pickle.dump(kpc_index_dict, f)
with open('kpc_c09k/c09k_label_ind.pickle', 'wb') as f:
    pickle.dump(kpc_label_dict, f)

In [ ]:
label_text1 = label_text.copy()
label_text1['label'] = label_text1['label'].apply(lambda x: kpc_label_dict[x])
label_text1

In [ ]:
train_text = label_text1[['text', 'label']].loc[label_text['ad'] < test_split_date]
test_text = label_text1[['text', 'label']].loc[label_text['ad'] >= test_split_date]
print(len(label_text), len(train_text), len(test_text))  # 15084 9881 5203

In [ ]:
train_text.to_csv('kpc_c09k/train_C09K11_220715.txt', encoding='utf-8', sep='\t', index=False)
test_text.to_csv('kpc_c09k/test_C09K11_220715.txt', encoding='utf-8', sep='\t', index=False)

In [ ]:
# min(train_text['text'].str.len())

### Load Model

In [ ]:
# device = torch.device("cpu")  ## CPU
device = torch.device("cuda:0")  ## GPU
model_path = 'ksic_model'
model_path1 = 'c09k_model'

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size=768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(),
                              attention_mask=attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
bertmodel = torch.load(os.path.join(model_path, 'KSIC_KoBERT.pt'))  # 전체 모델을 통째로 불러옴, 클래스 선언 필수

In [ ]:
bertmodel.load_state_dict(
    torch.load(os.path.join(model_path, 'KSIC_model_state_dict.pt')))  # state_dict를 불러 온 후, 모델에 저장

In [ ]:
_, vocab = get_pytorch_kobert_model(cachedir=".cache")

In [ ]:
try:
    bertmodel = torch.load(os.path.join(model_path, 'KSIC_KoBERT.pt'))  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
    bertmodel.load_state_dict(
        torch.load(os.path.join(model_path, 'KSIC_model_state_dict.pt')))  # state_dict를 불러 온 후, 모델에 저장
    print('Using saved model')
    _, vocab = get_pytorch_kobert_model(cachedir=".cache")
    print('complete loading vocab')
except Exception as e:
    print(e)
    bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

In [ ]:
print(vocab)

### GluonNLP Toolkit
* GluonNLP Toolkit provides tools for building efficient data pipelines for NLP tasks.
* https://nlp.gluon.ai/api/modules/data.html#gluonnlp.data.BERTSentenceTransform
* class gluonnlp.data.BERTSPTokenizer(path, vocab, num_best=0, alpha=1.0, lower=True, max_input_chars_per_word=200)[source]¶
* https://nlp.gluon.ai/api/modules/data.html#gluonnlp.data.TSVDataset

In [ ]:
dataset_train = nlp.data.TSVDataset(".cache/train_H04W4_220511.txt", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(".cache/test_H04W4_220511.txt", field_indices=[0,1], num_discard_samples=1)
# 텍스트 내에 줄바꿈과 탭이 존재하여 오류 발생, 처리 필요

In [ ]:
print(dataset_test[:5])

### GluonNLP BERTSPTokenizer
* BERTSPTokenizer depends on the sentencepiece library.
* For multi-processing with BERTSPTokenizer, making an extra copy of the BERTSPTokenizer instance is recommended before using it.
* https://nlp.gluon.ai/api/data.html?highlight=bertsptokenizer#gluonnlp.data.BERTSPTokenizer

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
print(dataset_train[2][0])

In [ ]:
sample_txt = dataset_train[2][0][:256]
# sample_txt = 'GaN 전력 증폭기를 이용한 전력 오실레이터, 본 발명은 GaN(Gallium Nitride) 소자로 구성되며, 입력 신호의 전력을 증폭시켜 출력하는 GaN 전력 증폭기, 상기 GaN 전력 증폭기의 출력 신호의 일부를 피드백 신호로 제공하는 디렉셔널 커플러, 상기 디렉셔널 커플러에 의해서 제공되는 피드백 신호의 페이저를 가변시키는 페이저 시프터 및 상기 페이저 시프터에 의한 임피던스 부정합을 조정하며, 상기 GaN 전력 증폭기로 상기 피드백 신호를 전달하는 제 1 아이솔레이터를 포함하는 GaN 전력 증폭기를 이용한 전력 오실레이터가 제공된다.'
print(tok(sample_txt))

In [ ]:
print(dataset_train[162][0])

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
#         self.labels = [i[label_idx] for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 512
batch_size = 8
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 1000
learning_rate =  5e-5

#### max_len vs batch size
* Text examples should be mostly less than 512 tokens. Longer texts will be cut from the end to fit the sequence length specified in the model block.
* https://peltarion.com/knowledge-center/documentation/cheat-sheets/bert---text-classification-/-cheat-sheet
* Sequence length	Recommended max batch size
    * 64 - 64, 128 - 32, 256 - 16, 320 - 14, 384 - 12, 512 - 6

In [ ]:
data_train_id = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test_id = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
print(data_train_id.sentences[162])

In [ ]:
print(data_train_id.labels[:3])

* DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, *, prefetch_factor=2, persistent_workers=False)
* https://pytorch.org/docs/stable/data.html
* WARNING
    * After several iterations, the loader worker processes will consume the same amount of CPU memory as the parent process for all Python objects in the parent process which are accessed from the worker processes. This can be problematic if the Dataset contains a lot of data (e.g., you are loading a very large list of filenames at Dataset construction time) and/or you are using a lot of workers (overall memory usage is number of workers * size of parent process). The simplest workaround is to replace Python objects with non-refcounted representations such as Pandas, Numpy or PyArrow objects. Check out issue #13246 for more details on why this occurs and example code for how to workaround these problems.

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train_id, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test_id, batch_size=batch_size, num_workers=5, shuffle=True)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, num_classes=5, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# help(nn.CrossEntropyLoss)
# help(torch)

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
print(time.strftime('%l:%M%p %Z on %b %d, %Y')) # ' 1:36PM EDT on Oct 18, 2010'
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
print(time.strftime('%l:%M%p %Z on %b %d, %Y')) # ' 1:36PM EDT on Oct 18, 2010'

In [ ]:
test_dataloader2 = torch.utils.data.DataLoader(data_test_id, batch_size=batch_size, num_workers=5)
test_acc = 0.0
results = []
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader2), total=len(test_dataloader2)):
    print(batch_id)
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    max_vals, max_indices = torch.max(out, 1)
    results.extend(max_indices.tolist())
    test_acc += calc_accuracy(out, label)
    print("test acc {}".format(test_acc / (batch_id+1)))
print(results)
#     print('label: ', label)    

In [ ]:
print(max_indices)
print(max_indices.tolist())


In [ ]:
m = nn.Softmax(dim=1)
exp = m(out)
max_vals, max_indices = torch.max(out, 1)
print(max_indices)

In [ ]:
# https://dacon.io/en/competitions/official/235747/codeshare/3082?page=1&dtype=recent
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
# 5. 모델 학습 과정 표시하기


fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(loss.long().to(device), 'y', label='train loss')
# loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(train_acc.long().to(device), 'b', label='train_acc')
acc_ax.plot(test_acc.long().to(device), 'g', label='test_acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()


In [ ]:
# out = model(token_ids, valid_length, segment_ids)
# print(out)

In [ ]:
# 100%
# 2344/2344 [10:52<00:00, 3.82it/s]
# epoch 1 batch id 1 loss 0.7244999408721924 train acc 0.453125
# epoch 1 batch id 201 loss 0.4514557123184204 train acc 0.5766480099502488
# epoch 1 batch id 401 loss 0.44362887740135193 train acc 0.6851231296758105
# epoch 1 batch id 601 loss 0.5137903094291687 train acc 0.734426996672213
# epoch 1 batch id 801 loss 0.43659207224845886 train acc 0.7636352996254682
# epoch 1 batch id 1001 loss 0.3066664934158325 train acc 0.7802978271728271
# epoch 1 batch id 1201 loss 0.3117211163043976 train acc 0.7930890924229809
# epoch 1 batch id 1401 loss 0.30558276176452637 train acc 0.8026075124910778
# epoch 1 batch id 1601 loss 0.3304724097251892 train acc 0.8104992973141787
# epoch 1 batch id 1801 loss 0.25170841813087463 train acc 0.8164995141588006
# epoch 1 batch id 2001 loss 0.2823370695114136 train acc 0.8224481509245377
# epoch 1 batch id 2201 loss 0.3238280713558197 train acc 0.8274292935029532
# epoch 1 train acc 0.8307713843856656
# 100%
# 782/782 [01:06<00:00, 13.16it/s]
# epoch 1 test acc 0.8835118286445013
# 100%
# 2344/2344 [10:52<00:00, 3.81it/s]
# epoch 2 batch id 1 loss 0.4955632984638214 train acc 0.828125
# epoch 2 batch id 201 loss 0.2225867211818695 train acc 0.8816075870646766
# epoch 2 batch id 401 loss 0.33707231283187866 train acc 0.8836502493765586
# epoch 2 batch id 601 loss 0.39554905891418457 train acc 0.887115224625624
# epoch 2 batch id 801 loss 0.30988579988479614 train acc 0.8883426966292135
# epoch 2 batch id 1001 loss 0.28933045268058777 train acc 0.8911713286713286
# epoch 2 batch id 1201 loss 0.24474024772644043 train acc 0.8932269983347211
# epoch 2 batch id 1401 loss 0.1908964067697525 train acc 0.8957218058529621
# epoch 2 batch id 1601 loss 0.23474052548408508 train acc 0.89772993441599
# epoch 2 batch id 1801 loss 0.1599130779504776 train acc 0.8997171710161022
# epoch 2 batch id 2001 loss 0.20312610268592834 train acc 0.9019865067466267
# epoch 2 batch id 2201 loss 0.2386036366224289 train acc 0.9033251930940481
# epoch 2 train acc 0.904759047923777
# 100%
# 782/782 [01:06<00:00, 11.77it/s]
# epoch 2 test acc 0.8906449808184144
# 100%
# 2344/2344 [10:51<00:00, 3.81it/s]
# epoch 3 batch id 1 loss 0.3390277922153473 train acc 0.875
# epoch 3 batch id 201 loss 0.15983553230762482 train acc 0.9240516169154229
# epoch 3 batch id 401 loss 0.14856880903244019 train acc 0.9265118453865336
# epoch 3 batch id 601 loss 0.2642267644405365 train acc 0.9280366056572379
# epoch 3 batch id 801 loss 0.1951659917831421 train acc 0.93020443196005
# epoch 3 batch id 1001 loss 0.26453569531440735 train acc 0.9319586663336663
# epoch 3 batch id 1201 loss 0.1282612681388855 train acc 0.9340523522064946
# epoch 3 batch id 1401 loss 0.1837957799434662 train acc 0.9360501427551748
# epoch 3 batch id 1601 loss 0.14137345552444458 train acc 0.9374414428482198
# epoch 3 batch id 1801 loss 0.09849003702402115 train acc 0.9387493059411438
# epoch 3 batch id 2001 loss 0.15634101629257202 train acc 0.9401080709645178
# epoch 3 batch id 2201 loss 0.1982114464044571 train acc 0.9410495229441163
# epoch 3 train acc 0.9420039640216155
# 100%
# 782/782 [01:06<00:00, 11.77it/s]
# epoch 3 test acc 0.8969988810741688
# 100%
# 2344/2344 [10:52<00:00, 3.82it/s]
# epoch 4 batch id 1 loss 0.388761043548584 train acc 0.875
# epoch 4 batch id 201 loss 0.06205718219280243 train acc 0.9593439054726368
# epoch 4 batch id 401 loss 0.06854245811700821 train acc 0.9596711346633416
# epoch 4 batch id 601 loss 0.23485814034938812 train acc 0.9600925540765392
# epoch 4 batch id 801 loss 0.1342923790216446 train acc 0.9612008426966292
# epoch 4 batch id 1001 loss 0.1908232569694519 train acc 0.9621472277722277
# epoch 4 batch id 1201 loss 0.11091630905866623 train acc 0.9632597835137385
# epoch 4 batch id 1401 loss 0.10650145262479782 train acc 0.9642219842969307
# epoch 4 batch id 1601 loss 0.08601253479719162 train acc 0.9649242660836976
# epoch 4 batch id 1801 loss 0.057230446487665176 train acc 0.9656093836757357
# epoch 4 batch id 2001 loss 0.07411567866802216 train acc 0.9665011244377811
# epoch 4 batch id 2201 loss 0.1597125381231308 train acc 0.9671456156292594
# epoch 4 train acc 0.9676701151877133
# 100%
# 782/782 [01:06<00:00, 11.77it/s]
# epoch 4 test acc 0.8980778452685422
# 100%
# 2344/2344 [10:52<00:00, 3.81it/s]
# epoch 5 batch id 1 loss 0.3727969229221344 train acc 0.890625
# epoch 5 batch id 201 loss 0.02794063650071621 train acc 0.9752798507462687
# epoch 5 batch id 401 loss 0.024620698764920235 train acc 0.9767378428927681
# epoch 5 batch id 601 loss 0.15002880990505219 train acc 0.9765495008319468
# epoch 5 batch id 801 loss 0.05448848009109497 train acc 0.9766112671660424
# epoch 5 batch id 1001 loss 0.08006531745195389 train acc 0.9770541958041958
# epoch 5 batch id 1201 loss 0.04451199620962143 train acc 0.9775317443796836
# epoch 5 batch id 1401 loss 0.08561042696237564 train acc 0.9780625446109922
# epoch 5 batch id 1601 loss 0.026716381311416626 train acc 0.9783533728919426
# epoch 5 batch id 1801 loss 0.02442212402820587 train acc 0.9787357717934481
# epoch 5 batch id 2001 loss 0.0197431817650795 train acc 0.9790339205397302
# epoch 5 batch id 2201 loss 0.03802771866321564 train acc 0.9792565879145843
# epoch 5 train acc 0.9794199729806597
# 100%
# 782/782 [01:06<00:00, 11.78it/s]
# epoch 5 test acc 0.8974384590792839